In [1]:
# 📌 Cell 1: Imports and Constants
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from collections import Counter

2025-03-24 19:36:43.868834: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-24 19:36:44.001784: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742825204.062508   11051 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742825204.082213   11051 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-24 19:36:44.179571: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# 📌 Cell 2: Constants and Paths
NUM_CROPS = 9
NUM_DISEASES = 33

MODEL_PATH = "/home/srivatsa/Multiple_Crop_Disease_Detection/Models/DANN/dann_model.keras"
TEST_TFRECORD_PATH = "/home/srivatsa/Multiple_Crop_Disease_Detection/Dataset/PlantVillage_Structured/AUG_TFRecord/split/test.tfrecord"
REPORT_SAVE_DIR = "reports/DANN_Model/Evaluation"
os.makedirs(REPORT_SAVE_DIR, exist_ok=True)

In [3]:
# 📌 Cell 3: TFRecord Parsing and Dataset Loader
feature_description = {
    "image": tf.io.FixedLenFeature([], tf.string),
    "crop": tf.io.FixedLenFeature([], tf.int64),
    "disease": tf.io.FixedLenFeature([], tf.int64)
}

def parse_example(example_proto):
    example = tf.io.parse_single_example(example_proto, feature_description)
    image = tf.image.decode_jpeg(example['image'], channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0
    crop = example['crop']
    disease = example['disease']
    return image, (crop, disease)

def load_dataset(path, batch_size=64):
    dataset = tf.data.TFRecordDataset(path)
    dataset = dataset.map(parse_example)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [4]:
# 📌 Cell 4: Load Model and Dataset
# Redefine custom loss function locally instead of importing

def weighted_categorical_crossentropy(weights_array):
    weights_tensor = tf.constant(weights_array, dtype=tf.float32)
    def loss_fn(y_true, y_pred):
        base_loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
        true_labels = tf.argmax(y_true, axis=-1)
        sample_weights = tf.gather(weights_tensor, true_labels)
        weighted_loss = base_loss * sample_weights
        return tf.reduce_mean(weighted_loss)
    return loss_fn

crop_weights_array = np.ones(NUM_CROPS)
disease_weights_array = np.ones(NUM_DISEASES)

model = tf.keras.models.load_model(
    MODEL_PATH,
    custom_objects={
        'loss_fn': weighted_categorical_crossentropy
    },
    compile=False
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss={
        "crop_output": weighted_categorical_crossentropy(crop_weights_array),
        "disease_output": weighted_categorical_crossentropy(disease_weights_array)
    },
    loss_weights={"crop_output": 1.0, "disease_output": 2.0},
    metrics={"crop_output": "accuracy", "disease_output": "accuracy"}
)

test_dataset = load_dataset(TEST_TFRECORD_PATH)

I0000 00:00:1742825210.598825   11051 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [5]:
# 📌 Cell 5: Prediction & Ground Truth Accumulation
crop_preds, disease_preds = [], []
crop_labels, disease_labels = [], []

for batch in test_dataset:
    images, (true_crop, true_disease) = batch
    outputs = model.predict(images, verbose=0)
    pred_crop = outputs[0]
    pred_disease = outputs[1]
    crop_preds.extend(np.argmax(pred_crop, axis=1))
    disease_preds.extend(np.argmax(pred_disease, axis=1))
    crop_labels.extend(true_crop.numpy())
    disease_labels.extend(true_disease.numpy())


2025-03-24 19:36:58.765951: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:370] TFRecordDataset `buffer_size` is unspecified, default to 262144
I0000 00:00:1742825221.514378   11164 service.cc:148] XLA service 0x7fcf34003360 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742825221.514785   11164 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-03-24 19:37:01.652469: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742825222.272643   11164 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-24 19:37:03.320568: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2298', 16 bytes spill stores, 16 bytes spill loads

2025-03-24 19:37:03.379622: I exter

In [6]:
# 📌 Cell 6: Confusion Matrices
crop_cm = confusion_matrix(crop_labels, crop_preds)
disease_cm = confusion_matrix(disease_labels, disease_preds)

plt.figure(figsize=(10, 8))
sns.heatmap(crop_cm, annot=True, fmt="d", cmap="Blues")
plt.title("Crop Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.savefig(os.path.join(REPORT_SAVE_DIR, "crop_confusion_matrix.png"))
plt.close()

plt.figure(figsize=(14, 10))
sns.heatmap(disease_cm, annot=False, cmap="Blues")
plt.title("Disease Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.savefig(os.path.join(REPORT_SAVE_DIR, "disease_confusion_matrix.png"))
plt.close()

In [7]:
# 📌 Cell 7: Classification Reports
crop_report = classification_report(crop_labels, crop_preds, digits=4)
disease_report = classification_report(disease_labels, disease_preds, digits=4)

with open(os.path.join(REPORT_SAVE_DIR, "classification_report.txt"), "w") as f:
    f.write("===== Crop Classification Report =====\n")
    f.write(crop_report + "\n\n")
    f.write("===== Disease Classification Report =====\n")
    f.write(disease_report)

In [8]:
# 📌 Cell 8: Save Class Mappings
crop_class_mapping = {0: 'Apple', 1: 'Bell_Pepper', 2: 'Cherry', 3: 'Corn', 4: 'Grape', 5: 'Peach', 6: 'Potato', 7: 'Strawberry', 8: 'Tomato'}
disease_class_mapping = {i: f'Disease_{i}' for i in range(NUM_DISEASES)}  # Customize if needed

with open(os.path.join(REPORT_SAVE_DIR, "class_mappings.txt"), "w") as f:
    f.write("Crop Label Mapping:\n")
    for k, v in crop_class_mapping.items():
        f.write(f"{k}: {v}\n")
    f.write("\nDisease Label Mapping:\n")
    for k, v in disease_class_mapping.items():
        f.write(f"{k}: {v}\n")

print("✅ Confusion Matrices, Reports and Class mappings saved!")

✅ Confusion Matrices, Reports and Class mappings saved!
